# 3D Convonlutional Neural Network

#### Install dependencies

In [ ]:
# Use this to install all dependencies on the local machine.
# This is not needed for running on Google Colab.

# %pip install tensorflow[and-cuda] keras gdown

#### Prepare imports

In [ ]:
import tensorflow # This is the ML library we will use
from tensorflow.keras.models import Sequential # This is the Sequential layer that runs all the sublayers inside
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D # These are the layer we will use in the model
from tensorflow.keras.utils import to_categorical # This is used to transform labels to categorical format
import h5py # This is used to work with h5 files
import numpy as np # This is used to handle arrays of data
import matplotlib.pyplot as plt # This is to load plotting functions
import gdown # This is used to download data from Google Drive
import os # This is used to work with file system

#### Prepare paths of the data and models

In [ ]:
ROOT = "./data"
PRETRAINED_MODEL_PATH = ROOT + '/pretrained_3d_cnn.h5'
MODEL_PATH = ROOT + '/3d_cnn.keras'
DATSET_PATH = ROOT + '/full_dataset_vectors.h5'

os.makedirs(ROOT, exist_ok=True)

#### Download the 3D MNIST dataset and the pretrained model files

In [ ]:
gdown.download(
    'https://drive.google.com/uc?id=1-jDaDuk5ePsRtS1TU85zJ2euBUPc05mU',
    DATSET_PATH,
    quiet=False
)
gdown.download(
    'https://drive.google.com/uc?id=1EG5g_xYCH7gtV6Zdrt1F_h6AQRQHzyGd',
    PRETRAINED_MODEL_PATH,
    quiet=False
)

#### Model configuration

In [ ]:
batch_size = 100 # Number of images to process at once 
no_epochs = 10 # Numbe of epochs to train the model
learning_rate = 0.001 # Rate of change of model parameters each training step
no_classes = 10 # Number of classes in the dataset
validation_split = 0.2 # Fraction of data to be used for validation 
verbosity = 1 # How verbose the training should be

#### Helper functions

In [ ]:
# Convert 1D vector into 3D values, provided by the 3D MNIST authors at
# https://www.kaggle.com/daavoo/3d-mnist
def array_to_color(array, cmap="Oranges"):
  s_m = plt.cm.ScalarMappable(cmap=cmap)
  return s_m.to_rgba(array)[:,:-1]

# Reshape data into format that can be handled by Conv3D layers.
# Courtesy of Sam Berglin; Zheming Lian; Jiahui Jang - University of Wisconsin-Madison
# Report - https://github.com/sberglin/Projects-and-Papers/blob/master/3D%20CNN/Report.pdf
# Code - https://github.com/sberglin/Projects-and-Papers/blob/master/3D%20CNN/network_final_version.ipynb
def rgb_data_transform(data):
  data_t = []
  for i in range(data.shape[0]):
    data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
  return np.asarray(data_t, dtype=np.float32)

#### Load the dataset from the downloaded data file

In [ ]:
# Load the HDF5 data file
with h5py.File(DATSET_PATH, "r") as hf:

    # Split the data into training/test features/targets
    X_train = hf["X_train"][:]
    targets_train = hf["y_train"][:]
    X_test = hf["X_test"][:]
    targets_test = hf["y_test"][:]

    # Determine sample shape
    sample_shape = (16, 16, 16, 3)

    # Reshape data into 3D format
    X_train = rgb_data_transform(X_train)
    X_test = rgb_data_transform(X_test)

    # Convert target vectors to categorical targets
    targets_train = to_categorical(targets_train).astype(np.int64)
    targets_test = to_categorical(targets_test).astype(np.int64)

#### Create the model

Use model.add function to add the following layers:

Conv3D layer with 32 output channels, kernel size of 3x3x3, relu activation function, he_uniform function for kernel initializer, input_shape=sample_shape

MaxPooling3D layer with 2x2x2 size

Conv3D layer with 64 output channels, kernel size of 3x3x3, relu activation function, he_uniform function for kernel initializer

MaxPooling3D layer with 2x2x2 size

Flatten layer

Dense layer with 256 output dimensions, relu activation function, he_uniform function for kernel initializer

Dense layer with `no_classes` output dimensions and softmax activation function

<details>
<summary>Solution</summary>
    <code>
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))<br>
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))<br>
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))<br>
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))<br>
    model.add(Flatten())<br>
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))<br>
    model.add(Dense(no_classes, activation='softmax'))<br>
    </code>
</details>

In [ ]:
# Create the model
model = Sequential()
...
model.summary()

#### Train the model

In [ ]:
# Compile the model
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

# Fit model to data 
history = model.fit(X_train, targets_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split)
model.save(MODEL_PATH)

#### Load the pretrained model weights into a separate model

In [ ]:
pretrained_model = tensorflow.keras.models.load_model(PRETRAINED_MODEL_PATH)
pretrained_model.summary()

#### Test the models

In [ ]:
# Generate generalization metrics
score_pretrained = pretrained_model.evaluate(X_test, targets_test, verbose=0)
score = model.evaluate(X_test, targets_test, verbose=0)
print(f'Pretrained model. Test loss: {score_pretrained[0]} / Test accuracy: {score_pretrained[1]}')
print(f'Trained model. Test loss: {score[0]} / Test accuracy: {score[1]}')

#### Plot losses

In [ ]:

# Plot history: Categorical crossentropy & Accuracy
plt.plot(history.history['loss'], label='Categorical crossentropy (training data)')
plt.plot(history.history['val_loss'], label='Categorical crossentropy (validation data)')
plt.plot(history.history['accuracy'], label='Accuracy (training data)')
plt.plot(history.history['val_accuracy'], label='Accuracy (validation data)')
plt.title('Model performance for 3D MNIST Keras Conv3D example')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()